# Sistema de Correlação de desbalanceamento de filtro de capacitores

In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
DATA_PATH = 'data/'
FASE = 'A'
MEASURES = 'MEDIDA_CAPACITANCIA_FASE_A.txt'
SERIE_AND_POSITION = 'ramo_serie_posicao_fase_A.csv'

# Pré-Processamento

## Carregamento das configurações iniciais

Arquivo de configurações iniciais contendo as contantes e restrições para o sistema. 

In [3]:
with open('data/config.json', 'r') as file:
    config = json.load(file)

In [4]:
config

{'constants': {'capacit_nom_capacitor_uF': 24.3,
  'capacit_nom_fase_uF': 3.24,
  'temp_ref_C': 20,
  'tensao_nom_kV': 530,
  'frequencia_Hz': 60},
 'constraints': {'tolerancia_diferenca': 0.001,
  'tolerancia_diferenca_pernas': 0.001,
  'tolerancia_var_capacit_fase': 5,
  'val_admit_capacit_capacitor_uF': [23.085, 25.515],
  'corrente_desbalanco_alarme_A': 0.17,
  'tensao_nom_sistema_kV': 530}}

## Carregamento dos dados

### Medidas de campo

In [5]:
raw_data = pd.read_csv(DATA_PATH + MEASURES, delimiter='\t', decimal=',')
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 9 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Serial     120 non-null    object 
 1   Date Time  120 non-null    object 
 2   ref(uF)    120 non-null    float64
 3   (uF)       120 non-null    float64
 4   (uF).1     120 non-null    float64
 5   (uF).2     120 non-null    float64
 6   (uF).3     120 non-null    float64
 7   (uF).4     120 non-null    float64
 8   (C)        120 non-null    int64  
dtypes: float64(6), int64(1), object(2)
memory usage: 8.6+ KB


### Relação Rack, Ramo, Série e Posição

In [6]:
serie_position = pd.read_csv(DATA_PATH + SERIE_AND_POSITION, delimiter=',')
serie_position.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 5 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   rack                     120 non-null    object 
 1   ramo                     120 non-null    object 
 2   posicao_eln              120 non-null    int64  
 3   num_serie                120 non-null    int64  
 4   capacitancia_fabrica_uF  120 non-null    float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.8+ KB


In [7]:
# Mapa para relação de pernas
pernas = {
    'A1' : 'p1',
    'A2' : 'p1',
    'B1' : 'p2',
    'B2' : 'p2',
    'A3' : 'p3',
    'A4' : 'p3',
    'B3' : 'p4',
    'B4' : 'p4'
}

serie_position['perna'] = serie_position['ramo'].map(pernas)
serie_position

,rack,ramo,posicao_eln,num_serie,capacitancia_fabrica_uF,perna
0,R1,A1,1,923,24.87,p1
1,R1,A1,2,925,24.88,p1
2,R1,A2,3,924,24.86,p1
3,R1,A2,4,919,24.95,p1
4,R1,A2,5,557,24.57,p1
...,...,...,...,...,...,...
115,R2,B4,84,991,24.67,p4
116,R2,B4,85,1007,24.78,p4
117,R2,B4,86,993,24.75,p4
118,R2,B3,87,1001,24.81,p4


## Tratamento dos dados

In [8]:
# Cálculo da capacitância média
raw_data['capacitancia_campo_uF'] = np.mean([raw_data['(uF)'], raw_data['(uF).1'], raw_data['(uF).2']], axis=0)
raw_data.head()

,Serial,Date Time,ref(uF),(uF),(uF).1,(uF).2,(uF).3,(uF).4,(C),capacitancia_campo_uF
0,,22/12/2020 08:09:18,0.0,24.66,24.55,24.55,0.0,0.0,25,24.586667
1,,22/12/2020 08:10:12,0.0,24.58,24.66,24.50,0.0,0.0,26,24.580000
2,,22/12/2020 08:10:48,0.0,24.63,24.69,24.60,0.0,0.0,26,24.640000
3,,22/12/2020 08:11:24,0.0,24.71,24.76,24.64,0.0,0.0,27,24.703333
4,,22/12/2020 08:12:20,0.0,24.43,24.50,24.31,0.0,0.0,27,24.413333


In [9]:
# Criação de um dataframe com as medições de campo tratada
field_measurements = pd.DataFrame()
field_measurements = raw_data[['Serial', 'Date Time', '(C)']]

In [10]:
# Média das medições de capacitância
field_measurements.loc[:, 'capacitancia_campo_uF'] = np.mean([raw_data['(uF)'], raw_data['(uF).1'], raw_data['(uF).2']], axis=0)

/tmp/ipykernel_50862/1349495368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  field_measurements.loc[:, 'capacitancia_campo_uF'] = np.mean([raw_data['(uF)'], raw_data['(uF).1'], raw_data['(uF).2']], axis=0)


In [11]:
# Renomeando as colunas
field_measurements = field_measurements.rename(columns={'Serial': 'serial', 'Date Time': 'date_time', '(C)': 'temperatura_capacitor_C'})

In [12]:
# Atribuindo o serial como índice
field_measurements.loc[:, 'serial'] = field_measurements.index + 1
field_measurements

,serial,date_time,temperatura_capacitor_C,capacitancia_campo_uF
0,1,22/12/2020 08:09:18,25,24.586667
1,2,22/12/2020 08:10:12,26,24.580000
2,3,22/12/2020 08:10:48,26,24.640000
3,4,22/12/2020 08:11:24,27,24.703333
4,5,22/12/2020 08:12:20,27,24.413333
...,...,...,...,...
115,116,22/12/2020 09:43:53,35,24.553333
116,117,22/12/2020 09:44:39,35,24.660000
117,118,22/12/2020 09:45:10,35,24.600000
118,119,22/12/2020 09:45:52,35,24.630000


In [13]:
# Junção dos dataframes
field_measurements_merged = pd.merge(field_measurements, serie_position, left_on='serial', right_on='posicao_eln', how='inner')

In [14]:
field_measurements_merged.head(5)

,serial,date_time,temperatura_capacitor_C,capacitancia_campo_uF,rack,ramo,posicao_eln,num_serie,capacitancia_fabrica_uF,perna
0,1,22/12/2020 08:09:18,25,24.586667,R1,A1,1,923,24.87,p1
1,2,22/12/2020 08:10:12,26,24.580000,R1,A1,2,925,24.88,p1
2,3,22/12/2020 08:10:48,26,24.640000,R1,A2,3,924,24.86,p1
3,4,22/12/2020 08:11:24,27,24.703333,R1,A2,4,919,24.95,p1
4,5,22/12/2020 08:12:20,27,24.413333,R1,A2,5,557,24.57,p1


# Ajustes e conformidade

## Definição de parâmetros : contantes e restrições

In [15]:
contants = {
    'capacit_nom_uF': 24.3, # Capacitância nominal em uF
    'temp_ref_C': 20, # Temperatura de referência em graus Celsius
}

constraints = {
    'delta_capacit_nominal': [-5, 5], # Delta percentual da capacitância nominal (em %)
}

constraints['valores_admitidos'] = [
    contants['capacit_nom_uF'] * (1 + constraints['delta_capacit_nominal'][0] / 100), # capacit_nom_uF - delta_capacit_nominal(%)
    contants['capacit_nom_uF'] * (1 + constraints['delta_capacit_nominal'][1] / 100), # capacit_nom_uF + delta_capacit_nominal(%)
] # Valores admitidos para a capacitância

In [16]:
constraints

{'delta_capacit_nominal': [-5, 5], 'valores_admitidos': [23.085, 25.515]}

## Delta de capacitância em relação ao valor de referência

$$\Delta C = \frac{Capacitancia - Capacitancia_{nominal}}{Capacitancia_{nominal}}$$

In [17]:
# Delta percentual capacitancia de fabrica e campo
field_measurements_merged['delta_capacitancia_fabrica_%'] = 100*(field_measurements_merged['capacitancia_fabrica_uF'] - config['constants']['capacit_nom_capacitor_uF'])/config['constants']['capacit_nom_capacitor_uF']
field_measurements_merged['delta_capacitancia_campo_%'] = 100*(field_measurements_merged['capacitancia_campo_uF'] - config['constants']['capacit_nom_capacitor_uF'])/config['constants']['capacit_nom_capacitor_uF']


In [18]:
# Delta percentual capacitancia de fabrica e campo
field_measurements_merged['delta_capacitancia_fabrica_%'] = 100*(field_measurements_merged['capacitancia_fabrica_uF'] - contants['capacit_nom_uF'])/contants['capacit_nom_uF']
field_measurements_merged['delta_capacitancia_campo_%'] =   100*(field_measurements_merged['capacitancia_campo_uF'] - contants['capacit_nom_uF'])/contants['capacit_nom_uF']


## Valor da capacitância de campo ajustada devido à temperatura

$$Capacitancia_{ajustada} = \frac{Capacitancia_{medida}} {(1 + \alpha \times (Temp_{medida} - Temp_{referencia}))}$$

Sendo $\alpha$ o coeficiente de temperatura do material do capacitor, $Temp_{medida}$ a temperatura medida e $Temp_{referencia}$ a temperatura de referência.

Para este sistema, o valor de $\alpha = -0.0004$.

In [19]:
alpha = -0.0004
field_measurements_merged['capacitancia_campo_ajustada_uF'] =  field_measurements_merged['capacitancia_campo_uF'] / (1 + alpha * (field_measurements_merged['temperatura_capacitor_C'] - config['constants']['temp_ref_C']))

In [20]:
field_measurements_merged.head()

,serial,date_time,temperatura_capacitor_C,capacitancia_campo_uF,rack,ramo,posicao_eln,num_serie,capacitancia_fabrica_uF,perna,delta_capacitancia_fabrica_%,delta_capacitancia_campo_%,capacitancia_campo_ajustada_uF
0,1,22/12/2020 08:09:18,25,24.586667,R1,A1,1,923,24.87,p1,2.345679,1.179698,24.635939
1,2,22/12/2020 08:10:12,26,24.580000,R1,A1,2,925,24.88,p1,2.386831,1.152263,24.639134
2,3,22/12/2020 08:10:48,26,24.640000,R1,A2,3,924,24.86,p1,2.304527,1.399177,24.699278
3,4,22/12/2020 08:11:24,27,24.703333,R1,A2,4,919,24.95,p1,2.674897,1.659808,24.772697
4,5,22/12/2020 08:12:20,27,24.413333,R1,A2,5,557,24.57,p1,1.111111,0.466392,24.481883


In [21]:
field_measurements_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 13 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   serial                          120 non-null    object 
 1   date_time                       120 non-null    object 
 2   temperatura_capacitor_C         120 non-null    int64  
 3   capacitancia_campo_uF           120 non-null    float64
 4   rack                            120 non-null    object 
 5   ramo                            120 non-null    object 
 6   posicao_eln                     120 non-null    int64  
 7   num_serie                       120 non-null    int64  
 8   capacitancia_fabrica_uF         120 non-null    float64
 9   perna                           120 non-null    object 
 10  delta_capacitancia_fabrica_%    120 non-null    float64
 11  delta_capacitancia_campo_%      120 non-null    float64
 12  capacitancia_campo_ajustada_uF  120 

## Classificação de conformidade devido à capacitância de campo ajustada

A capacitância de campo ajustada é classificada como conforme se atender a condição de estar dentro da faixa de valores admitidos, já definidos nas restrições.

$$\text{Conformidade} = \begin{cases}
    \text{Conforme (C)} & \text{se } Capacitancia_{ajustada} \in (Capacitancia_{min}, Capacitancia_{max}) \\
    \text{Não Conforme (NC)} & \text{se } Capacitancia_{ajustada} \notin (Capacitancia_{min}, Capacitancia_{max})
\end{cases}$$

Sendo que 
$$Capacitancia_{min} = Capacitancia_{nominal} \times (1 - \text{tolerancia})$$ 
e 
$$Capacitancia_{max} = Capacitancia_{nominal} \times (1 + \text{tolerancia})$$

Com $\text{tolerancia} = 5\%$.

   

In [22]:
field_measurements_merged['conformidade'] = np.where(
    (field_measurements_merged['capacitancia_campo_ajustada_uF'] > config['constraints']['val_admit_capacit_capacitor_uF'][0]) & 
    (field_measurements_merged['capacitancia_campo_ajustada_uF'] < config['constraints']['val_admit_capacit_capacitor_uF'][1]), 
    "C", 
    "NC"
)

## Reordenamento das colunas

In [61]:
field_measurements_merged.columns

Index(['date_time', 'rack', 'ramo', 'perna', 'posicao_eln', 'num_serie',
       'capacitancia_fabrica_uF', 'delta_capacitancia_fabrica_%',
       'capacitancia_campo_uF', 'delta_capacitancia_campo_%',
       'temperatura_capacitor_C', 'capacitancia_campo_ajustada_uF',
       'conformidade', 'INV_capacitancia_campo_ajustada_uF'],
      dtype='object')

In [24]:
field_measurements_merged = field_measurements_merged[
    [
    'date_time',
    'rack',
    'ramo',
    'perna',
    'posicao_eln',
    'num_serie',
    'capacitancia_fabrica_uF',
    'delta_capacitancia_fabrica_%',
    'capacitancia_campo_uF',
    'delta_capacitancia_campo_%',
    'temperatura_capacitor_C',
    'capacitancia_campo_ajustada_uF',
    'conformidade'
    ]
]

In [25]:
field_measurements_merged

,date_time,rack,ramo,perna,posicao_eln,num_serie,capacitancia_fabrica_uF,delta_capacitancia_fabrica_%,capacitancia_campo_uF,delta_capacitancia_campo_%,temperatura_capacitor_C,capacitancia_campo_ajustada_uF,conformidade
0,22/12/2020 08:09:18,R1,A1,p1,1,923,24.87,2.345679,24.586667,1.179698,25,24.635939,C
1,22/12/2020 08:10:12,R1,A1,p1,2,925,24.88,2.386831,24.580000,1.152263,26,24.639134,C
2,22/12/2020 08:10:48,R1,A2,p1,3,924,24.86,2.304527,24.640000,1.399177,26,24.699278,C
3,22/12/2020 08:11:24,R1,A2,p1,4,919,24.95,2.674897,24.703333,1.659808,27,24.772697,C
4,22/12/2020 08:12:20,R1,A2,p1,5,557,24.57,1.111111,24.413333,0.466392,27,24.481883,C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,22/12/2020 09:43:53,R1,B1,p2,116,1100,24.75,1.851852,24.553333,1.042524,35,24.701543,C
116,22/12/2020 09:44:39,R1,B1,p2,117,1089,24.82,2.139918,24.660000,1.481481,35,24.808853,C
117,22/12/2020 09:45:10,R1,B1,p2,118,1070,24.85,2.263374,24.600000,1.234568,35,24.748491,C
118,22/12/2020 09:45:52,R1,B2,p2,119,1073,24.93,2.592593,24.630000,1.358025,35,24.778672,C


# Cálculo dos parâmetros do sistema 

## Cálculo das capacitâncias

Nesta fase são calculadas as capacitâncias de :

### Ramo

Seja o conjunto de ramos $\{A_1, A_2, A_3, A_3, B_1, B_2, B_3, B_4\}$. Tomando o ramo $A_1$ como exemplo, a sua capacitância é dada em série por :

$$\frac{1}{C_{A1}} = \frac{1}{C_{1}} + \frac{1}{C_{2}} + \frac{1}{C_{3}} + \cdots + \frac{1}{C_{n}}$$

Sendo $C_{1}, C_{2}, C_{3}, \cdots, C_{n}$ as capacitâncias dos filtros do ramo $A_1$, e $C_{A1}$ a capacitância equivalente do ramo $A_1$.

Neste trabalho, usa-se a capacitância corrigida devido à temperatura ($C_{ajustada}$) dos capacitores unitários, para o cálculo da capacitância equivalente do ramo.

### Perna

Seja o conjunto de pernas $\{p_1, p_2, p_3, p_4\}$. Cada perna é composto por um par de ramos, assim: 

$p_1 = \{A_1, A_2\}$ , $p_2 = \{A_3, B_4\}$, $p_3 = \{B_1, B_2\}$, $p_4 = \{B_3, B_4\}$

A capacitância e equivalente de cada perna é dada em paralelo, assim: 

* $C_{p1} = C_{A1} + C_{A2}$
* $C_{p2} = C_{A3} + C_{B4}$
* $C_{p3} = C_{B1} + C_{B2}$
* $C_{p4} = C_{B3} + C_{B4}$

### Rack

Seja o conjunto de racks $\{R_1, R_2\}$. Cada rack é composto por um par de pernas, assim:

$R_1 = \{p_1, p_2\}$, $R_2 = \{p_3, p_4\}$

A capacitância equivalente de cada rack é dada em paralelo, assim:

* $C_{R1} = C_{p1} + C_{p2}$
* $C_{R2} = C_{p3} + C_{p4}$

### Fase

Seja o conjunto de fases $\{F_A, F_B\}$. Cada fase é composta por um par de racks, assim:

$F_A = \{R_1, R_2\}$, $F_B = \{R_3, R_4\}$

A capacitância equivalente de cada fase é dada em série, assim:

$$C_{FA} = \frac{C_{R1} * C_{R2}}{C_{R1} + C_{R2}}$$
$$C_{FB} = \frac{C_{R1} * C_{R2}}{C_{R1} + C_{R2}}$$



In [26]:
field_measurements_merged['INV_capacitancia_campo_ajustada_uF'] = 1/field_measurements_merged['capacitancia_campo_ajustada_uF']
field_measurements_merged[['capacitancia_campo_ajustada_uF', 'INV_capacitancia_campo_ajustada_uF']]
field_measurements_merged

,date_time,rack,ramo,perna,posicao_eln,num_serie,capacitancia_fabrica_uF,delta_capacitancia_fabrica_%,capacitancia_campo_uF,delta_capacitancia_campo_%,temperatura_capacitor_C,capacitancia_campo_ajustada_uF,conformidade,INV_capacitancia_campo_ajustada_uF
0,22/12/2020 08:09:18,R1,A1,p1,1,923,24.87,2.345679,24.586667,1.179698,25,24.635939,C,0.040591
1,22/12/2020 08:10:12,R1,A1,p1,2,925,24.88,2.386831,24.580000,1.152263,26,24.639134,C,0.040586
2,22/12/2020 08:10:48,R1,A2,p1,3,924,24.86,2.304527,24.640000,1.399177,26,24.699278,C,0.040487
3,22/12/2020 08:11:24,R1,A2,p1,4,919,24.95,2.674897,24.703333,1.659808,27,24.772697,C,0.040367
4,22/12/2020 08:12:20,R1,A2,p1,5,557,24.57,1.111111,24.413333,0.466392,27,24.481883,C,0.040847
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,22/12/2020 09:43:53,R1,B1,p2,116,1100,24.75,1.851852,24.553333,1.042524,35,24.701543,C,0.040483
116,22/12/2020 09:44:39,R1,B1,p2,117,1089,24.82,2.139918,24.660000,1.481481,35,24.808853,C,0.040308
117,22/12/2020 09:45:10,R1,B1,p2,118,1070,24.85,2.263374,24.600000,1.234568,35,24.748491,C,0.040407
118,22/12/2020 09:45:52,R1,B2,p2,119,1073,24.93,2.592593,24.630000,1.358025,35,24.778672,C,0.040357


In [27]:
def get_capacitancia_ramo(df, coluna_ramo):
    """
    Calcula a capacitância equivalente para cada ramo em um DataFrame.
    Args:
        df (pandas.DataFrame): DataFrame contendo os dados dos ramos e suas respectivas capacitâncias.
        coluna_ramo (str): Nome da coluna no DataFrame que identifica os ramos.
    Returns:
        dict: Um dicionário onde as chaves são os ramos e os valores são as capacitâncias equivalentes dos ramos.
    """
    
    # Obtém os ramos
    ramos = df[coluna_ramo].unique()
    # reordenar ramos
    ramos = np.sort(ramos)
    
    capacitancias_ramos = {}
    
    for ramo in ramos:
        ramo_data = df[df['ramo'] == ramo]
        # soma_inversos = (1 / ramo_data['capacitancia_campo_ajustada_uF']).sum()
        soma_inversos = np.sum(1 / ramo_data['capacitancia_campo_ajustada_uF'].values)
        capacitancias_ramos[ramo] = 1 / soma_inversos
    
    return capacitancias_ramos

def get_capacitancia_perna(capacitancias_ramos):
    """
    Calcula a capacitância de cada perna com base nas capacitâncias dos ramos.
    Args:
        capacitancias_ramos (dict): Um dicionário contendo as capacitâncias dos ramos.
            As chaves esperadas são 'A1', 'A2', 'A3', 'B1', 'B3' e 'B4'.
    Returns:
        dict: Um dicionário contendo as capacitâncias de cada perna.
            As chaves são 'p1', 'p2', 'p3' e 'p4' com seus respectivos valores de capacitância.
    """

    capacitancias_pernas = {
        'p1': capacitancias_ramos['A1'] + capacitancias_ramos['A2'],
        'p2': capacitancias_ramos['B1'] + capacitancias_ramos['B1'],
        'p3': capacitancias_ramos['A3'] + capacitancias_ramos['B4'],
        'p4': capacitancias_ramos['B3'] + capacitancias_ramos['B4']
    }
    
    return capacitancias_pernas

def get_capacitancia_racks(capacitancias_pernas):
        """
        Calcula a capacitância dos racks com base na capacitância das pernas.
        Args:
            capacitancias_pernas (dict): Um dicionário contendo os valores de capacitância das pernas.
            Espera-se que as chaves sejam 'p1', 'p2', 'p3' e 'p4'.
        Returns:
            dict: Um dicionário contendo os valores de capacitância dos racks.
            As chaves são 'R1' e 'R2', onde:
                - 'R1' é a soma de 'p1' e 'p2'
                - 'R2' é a soma de 'p3' e 'p4'
        """
        
        capacitancias_racks = {
            'R1': capacitancias_pernas['p1'] + capacitancias_pernas['p2'],
            'R2': capacitancias_pernas['p3'] + capacitancias_pernas['p4']
        }
        
        return capacitancias_racks

def get_capacitancia_fase(capacitancias_racks):
    """
    Calcula a capacitância de fase dada as capacitâncias de dois racks.
    Esta função calcula a capacitância equivalente de dois capacitores em série.
    A fórmula usada é: C_eq = (C1 * C2) / (C1 + C2), onde C1 e C2 são as 
    capacitâncias dos dois racks.
    
    Args:
        capacitancias_racks (dict): Um dicionário contendo as capacitâncias dos racks 
                                com as chaves 'R1' e 'R2'.
    Returns:
        float: A capacitância equivalente de fase.
    """
    
    capacitancia_fase = (capacitancias_racks['R1'] * capacitancias_racks['R2']) /  (capacitancias_racks['R1'] + capacitancias_racks['R2'])
    
    return capacitancia_fase



In [28]:
capacitancias_ramos = get_capacitancia_ramo(field_measurements_merged, 'ramo')
capacitancias_pernas = get_capacitancia_perna(capacitancias_ramos)
capacitancias_racks = get_capacitancia_racks(capacitancias_pernas)
capacitancia_fase = get_capacitancia_fase(capacitancias_racks)

In [29]:
print("Capacitâncias equivalentes dos ramos (μF):")
for ramo, cap in capacitancias_ramos.items():
    print(f"Ramo {ramo}: {cap}")

# Exibir resultados
print("\nCapacitâncias das pernas (μF):")
for perna, cap in capacitancias_pernas.items():
    print(f"Perna {perna}: {cap}")

print("\nCapacitâncias dos racks (μF):")
for rack, cap in capacitancias_racks.items():
    print(f"Rack {rack}: {cap}")

print(f"\nCapacitâncias da fase (μF): {capacitancia_fase}")


Capacitâncias equivalentes dos ramos (μF):
Ramo A1: 1.5431268098303932
Ramo A2: 1.5393547947488146
Ramo A3: 1.7651594622083038
Ramo A4: 1.7622254945638736
Ramo B1: 1.5447181344929917
Ramo B2: 1.5443203256952205
Ramo B3: 1.7664865239200103
Ramo B4: 1.7601458528110911

Capacitâncias das pernas (μF):
Perna p1: 3.0824816045792076
Perna p2: 3.0894362689859833
Perna p3: 3.525305315019395
Perna p4: 3.5266323767311016

Capacitâncias dos racks (μF):
Rack R1: 6.1719178735651905
Rack R2: 7.051937691750497

Capacitâncias da fase (μF): 3.291323023607443


## Cálculo das reatâncias capacitivas

As reatâncias capacitivas são calculadas a partir da frequência de operação do sistema, e da capacitância equivalente de cada segmento do sistema.

A equação que descreve a reatância capacitiva é dada por:

$$X_c = \frac{1}{2 \pi f C}$$

Sendo $f = 60 Hz$ a frequência de operação do sistema, $C$ a capacitância equivalente do segmento do sistema e $X_c$ a reatância capacitiva.

### Ramo

A reatância capacitiva do ramo é dada por:

$$X_{cA1} = \frac{1}{2 \pi f C_{ramo_i} \phi}$$

para $i \in \{A_1, A_2, A_3, A_4, B_1, B_2, B_3, B_4\}$

Sendo $\phi = 1E-6$ a constante uma constante de ajuste.

### Perna

Dado que as pernas são organizadas assim: 

$p_1 = \{A_1, A_2\}$ , $p_2 = \{B_1, B_2\}$, $p_3 = \{A_3, A_4\}$, $p_4 = \{B_3, B_4\}$

A reatância capacitiva equivalente de cada perna é a reatância em paralelo das reatâncias dos ramos que a compõem, assim:

$$X_{cpj} = \frac{C_{ramo'} * C_{ramo''}}{C_{ramo'} + C_{ramo''}}$$

para j $\in \{p_1, p_2, p_3, p_4\}$ e $C_{ramo'}, C_{ramo''}$ as reatâncias dos ramos que compõem a perna j.

### Rack 

Dado que os racks são organizados assim:

$R_1 = \{p_1, p_2\}$, $R_2 = \{p_3, p_4\}$

A reatância capacitiva equivalente de cada rack é a reatância em paralelo das reatâncias das pernas que o compõem, assim:

$$X_{cRk} = \frac{C_{perna'} * C_{perna''}}{C_{perna'} + C_{perna''}}$$

para k $\in \{R_1, R_2\}$ e $C_{perna'}, C_{perna''}$ as reatâncias das pernas que compõem o rack k.

### Fase

Dado que as fases são definidas pelo conjunto de racks $\{R_1, R_2\}$, a reatância capacitiva equivalente
da fase é a reatância em série das reatâncias dos racks que a compõem, assim:

$$X_{cF} = X_{cR1} + X_{cR2}$$






In [30]:
def get_reatancia_capacitiva_ramo(capacitancia, frequencia=60, phi=1e-6):
    """
    Calcula a reatância capacitiva para um ramo.
    Args:
        capacitancia (float): Capacitância em microfarads (uF).
        frequencia (float): Frequência em Hertz (Hz). Padrão é 60 Hz.
        phi (float): Constante de ajuste. Padrão é 1E-6.
    Returns:
        float: Reatância capacitiva em Ohms.
    """
    capacitancia_farad = capacitancia * phi  # Ajustar capacitância
    reatancia = 1 / (2 * np.pi * frequencia * capacitancia_farad)
    return reatancia

def get_reatancia_capacitiva_perna(reatancia_ramo1, reatancia_ramo2):
    """
    Calcula a reatância capacitiva para uma perna.
    Args:
        reatancia_ramo1 (float): Reatância do primeiro ramo em Ohms.
        reatancia_ramo2 (float): Reatância do segundo ramo em Ohms.
    Returns:
        float: Reatância capacitiva da perna em Ohms.
    """
    reatancia_perna = (reatancia_ramo1 * reatancia_ramo2) / (reatancia_ramo1 + reatancia_ramo2)
    return reatancia_perna

def get_reatancia_capacitiva_rack(reatancia_perna1, reatancia_perna2):
    """
    Calcula a reatância capacitiva para um rack.
    Args:
        reatancia_perna1 (float): Reatância da primeira perna em Ohms.
        reatancia_perna2 (float): Reatância da segunda perna em Ohms.
    Returns:
        float: Reatância capacitiva do rack em Ohms.
    """
    reatancia_rack = (reatancia_perna1 * reatancia_perna2) / (reatancia_perna1 + reatancia_perna2)
    return reatancia_rack

def get_reatancia_capacitiva_fase(reatancia_rack1, reatancia_rack2):
    """
    Calcula a reatância capacitiva para uma fase.
    Args:
        reatancia_rack1 (float): Reatância do primeiro rack em Ohms.
        reatancia_rack2 (float): Reatância do segundo rack em Ohms.
    Returns:
        float: Reatância capacitiva da fase em Ohms.
    """
    reatancia_fase = reatancia_rack1 + reatancia_rack2
    return reatancia_fase



In [31]:
capacitancias_ramos = get_capacitancia_ramo(field_measurements_merged, 'ramo')

# Reduz a precisão das capacitâncias para 3 casas decimais
capacitancias_ramos = {ramo: round(cap, 3) for ramo, cap in capacitancias_ramos.items()}

# Calcular reatâncias dos ramos
reatancias_ramos = {ramo: get_reatancia_capacitiva_ramo(cap) for ramo, cap in capacitancias_ramos.items()}

# Calcular reatâncias das pernas
reatancias_pernas = {
    'p1': get_reatancia_capacitiva_perna(reatancias_ramos['A1'], reatancias_ramos['A2']),
    'p2': get_reatancia_capacitiva_perna(reatancias_ramos['B1'], reatancias_ramos['B2']),
    'p3': get_reatancia_capacitiva_perna(reatancias_ramos['A3'], reatancias_ramos['A4']),
    'p4': get_reatancia_capacitiva_perna(reatancias_ramos['B3'], reatancias_ramos['B4'])
}

# Calcular reatâncias dos racks
reatancias_racks = {
    'R1': get_reatancia_capacitiva_rack(reatancias_pernas['p1'], reatancias_pernas['p2']),
    'R2': get_reatancia_capacitiva_rack(reatancias_pernas['p3'], reatancias_pernas['p4'])
}

# Calcular reatância da fase
reatancia_fase = get_reatancia_capacitiva_fase(reatancias_racks['R1'], reatancias_racks['R2'])

# Exibir resultados
print("\nReatâncias capacitivas dos ramos (Ohms):")
for ramo, reatancia in reatancias_ramos.items():
    print(f"Ramo {ramo}: {reatancia}")

print("\nReatâncias capacitivas das pernas (Ohms):")
for perna, reatancia in reatancias_pernas.items():
    print(f"Perna {perna}: {reatancia}")

print("\nReatâncias capacitivas dos racks (Ohms):")
for rack, reatancia in reatancias_racks.items():
    print(f"Rack {rack}: {reatancia}")

print("\nReatância capacitiva da fase (Ohms):")
print(f"Fase: {reatancia_fase}")


Reatâncias capacitivas dos ramos (Ohms):
Ramo A1: 1719.1071839694898
Ramo A2: 1723.5752988076172
Ramo A3: 1502.8795381670952
Ramo A4: 1505.4383569040422
Ramo B1: 1716.8818025015682
Ramo B2: 1717.9937725809084
Ramo B3: 1502.0285305010889
Ramo B4: 1507.1490823096153

Reatâncias capacitivas das pernas (Ohms):
Perna p1: 860.669170949034
Perna p2: 858.718803776278
Perna p3: 752.0789296469868
Perna p4: 752.2922248624286

Reatâncias capacitivas dos racks (Ohms):
Rack R1: 429.84644058741253
Rack R2: 376.0927810669109

Reatância capacitiva da fase (Ohms):
Fase: 805.9392216543234


## Cálculo das tensões e correntes

### Tensão tensão do sistema

A tensão do sistema é 

$$V_{sistema} = 530 kV$$

### Tensão de fase

A tensão de fase é dada por:

$$V_{fase} = \frac{V_{sistema}}{\sqrt{3}}$$

### Corrente de fase

A corrente de fase é dada por:

$$I_{fase} = \frac{V_{fase}}{X_{cF}}$$

### Tensão rack

A tensão de rack é dada pelo produto da corrente de fase pela reatância capacitiva do rack, assim:

$$V_{Rk} = I_{fase} \times X_{cRk}$$

Para k $\in \{R_1, R_2\}$

### Corrente perna

Dada que em um rack as pernas se relacionam em paralelo, a corrente de perna é dada pela razão entre tensão de rack e reatância capacitiva da perna, assim:

$$I_{p_i} = \frac{V_{Rk}}{X_{cp_i}}$$

para i $\in \{p_1, p_2, p_3, p_4\}$, k $\in \{R_1, R_2\}$ e $X_{cp_i}$ a reatância capacitiva da perna i.

### Corrente de desbalanceamento

Dada a relação entre as correntes de perna:

$I_p{1} = I_p{3}$ e $I_p{2} = I_p{4}$

A corrente de desbalanceamento é dada pela diferença entre as correntes de perna, assim:

$$I_{desbalanceamento} = |I_{p3} - I_{p1}| = |I_{p4} - I_{p2}|$$



In [32]:
# Constantes
V_sistema = config['constants']['tensao_nom_kV'] * 1000  # Tensão do sistema em Volts (530 kV)
V_fase = V_sistema / np.sqrt(3)  # Tensão de fase

def get_corrente_fase(V_fase, X_cF):
    """
    Calcula a corrente de fase.
    Args:
        V_fase (float): Tensão de fase em Volts.
        X_cF (float): Reatância capacitiva da fase em Ohms.
    Returns:
        float: Corrente de fase em Amperes.
    """
    return V_fase / X_cF

def get_tensao_rack(I_fase, X_cRk):
    """
    Calcula a tensão de rack.
    Args:
        I_fase (float): Corrente de fase em Amperes.
        X_cRk (float): Reatância capacitiva do rack em Ohms.
    Returns:
        float: Tensão de rack em Volts.
    """
    return I_fase * X_cRk

def get_corrente_perna(V_Rk, X_cp):
    """
    Calcula a corrente de perna.
    Args:
        V_Rk (float): Tensão de rack em Volts.
        X_cp (float): Reatância capacitiva da perna em Ohms.
    Returns:
        float: Corrente de perna em Amperes.
    """
    return V_Rk / X_cp

def get_corrente_desbalanceamento(I_p1, I_p2, I_p3, I_p4):
    """
    Calcula a corrente de desbalanceamento.
    Args:
        I_p1 (float): Corrente da perna 1 em Amperes.
        I_p2 (float): Corrente da perna 2 em Amperes.
        I_p3 (float): Corrente da perna 3 em Amperes.
        I_p4 (float): Corrente da perna 4 em Amperes.
    Returns:
        float: Corrente de desbalanceamento em Amperes.
    """

    I_desb_R1 = np.abs(I_p4 - I_p2)
    I_desb_R2 = np.abs(I_p3 - I_p1)
    
    return I_desb_R1, I_desb_R2



In [33]:
# Calcular corrente de fase
I_fase = get_corrente_fase(V_fase, reatancia_fase)

# Calcular tensões dos racks
tensoes_racks = {
    'R1': get_tensao_rack(I_fase, reatancias_racks['R1']),
    'R2': get_tensao_rack(I_fase, reatancias_racks['R2'])
}

# Calcular correntes das pernas
correntes_pernas = {
    'p1': get_corrente_perna(tensoes_racks['R1'], reatancias_pernas['p1']),
    'p2': get_corrente_perna(tensoes_racks['R1'], reatancias_pernas['p2']),
    'p3': get_corrente_perna(tensoes_racks['R2'], reatancias_pernas['p3']),
    'p4': get_corrente_perna(tensoes_racks['R2'], reatancias_pernas['p4'])
}

# Calcular corrente de desbalanceamento
I_desb_R1, I_desb_R2 = get_corrente_desbalanceamento(
    correntes_pernas['p1'],
    correntes_pernas['p2'],
    correntes_pernas['p3'],
    correntes_pernas['p4']
)

# Exibir resultados
print(f"Tensão do sistema (V): {V_sistema:.2f}")
print(f"Tensão de fase (V): {V_fase:.2f}")
print(f"Corrente de fase (A): {I_fase:.2f}")
print("\nCorrentes de desbalancemento dos racks")
print(f"Corrente desb Rack 1 (A): {I_desb_R1}")
print(f"Corrente desb Rack 2 (A): {I_desb_R2}")


print("\nTensões dos racks (V):")
for rack, tensao in tensoes_racks.items():
    print(f"Rack {rack}: {tensao:.2f}")

print("\nCorrentes das pernas (A):")
for perna, corrente in correntes_pernas.items():
    print(f"Perna {perna}: {corrente:.2f}")

Tensão do sistema (V): 530000.00
Tensão de fase (V): 305995.64
Corrente de fase (A): 379.68

Correntes de desbalancemento dos racks
Corrente desb Rack 1 (A): 0.24225627709981268
Corrente desb Rack 2 (A): 0.2422562770998411

Tensões dos racks (V):
Rack R1: 163202.30
Rack R2: 142793.34

Correntes das pernas (A):
Perna p1: 189.62
Perna p2: 190.05
Perna p3: 189.86
Perna p4: 189.81


In [34]:
field_measurements_merged.to_csv('data/field_measurements_merged.csv', index=False)

# Otimização do sistema desbalanceado

## Mais próximo de dar certo

In [ ]:
import numpy as np
import pandas as pd

class OtimizadorBancoCapacitores:
    def __init__(self, df: pd.DataFrame, config: dict, max_permutacoes: int):
        self.df = df.copy()
        self.df_original = df.copy()
        self.config = config
        self.max_permutacoes = max_permutacoes
        self.permutacoes = []
        self.capacitores_permutados = set()
        self.V_sistema = config['constants']['tensao_nom_kV'] * 1000  # Tensão do sistema em Volts
        self.V_fase = self.V_sistema / np.sqrt(3)  # Tensão de fase
        self.frequencia = config['constants']['frequencia_Hz']  # Frequência em Hz
        self.phi = 1e-6  # Constante de ajuste
        self.sucesso = False

    # Funções de cálculo
    def get_reatancia_capacitiva_ramo(self, capacitancia):
        capacitancia_farad = capacitancia * self.phi
        reatancia = 1 / (2 * np.pi * self.frequencia * capacitancia_farad)
        return reatancia

    def get_reatancia_capacitiva_perna(self, X_c_ramo1, X_c_ramo2):
        return (X_c_ramo1 * X_c_ramo2) / (X_c_ramo1 + X_c_ramo2)

    def get_reatancia_capacitiva_rack(self, X_c_perna1, X_c_perna2):
        return (X_c_perna1 * X_c_perna2) / (X_c_perna1 + X_c_perna2)

    def get_reatancia_capacitiva_fase(self, X_c_rack1, X_c_rack2):
        return X_c_rack1 + X_c_rack2

    def get_corrente_fase(self, X_cF):
        return self.V_fase / X_cF

    def get_tensao_rack(self, I_fase, X_cRk):
        return I_fase * X_cRk

    def get_corrente_perna(self, V_Rk, X_cp):
        return V_Rk / X_cp

    def get_corrente_desbalanceamento(self, I_p1, I_p2, I_p3, I_p4):
        I_desb_R1 = np.abs(I_p3 - I_p1)
        I_desb_R2 = np.abs(I_p4 - I_p2)
        return max(I_desb_R1, I_desb_R2)

    # Métodos de cálculo de capacitâncias
    def get_capacitancia_ramo(self, df, ramo):
        ramo_data = df[df['ramo'] == ramo]
        soma_inversos = np.sum(1 / ramo_data['capacitancia_campo_ajustada_uF'].values)
        return 1 / soma_inversos

    def get_capacitancia_perna(self, capacitancias_ramos, perna):
        if perna == 'p1':
            return capacitancias_ramos['A1'] + capacitancias_ramos['A2']
        elif perna == 'p2':
            return capacitancias_ramos['B1'] + capacitancias_ramos['B2']
        elif perna == 'p3':
            return capacitancias_ramos['A3'] + capacitancias_ramos['A4']
        elif perna == 'p4':
            return capacitancias_ramos['B3'] + capacitancias_ramos['B4']

    def get_capacitancia_rack(self, capacitancias_pernas, rack):
        if rack == 'R1':
            return capacitancias_pernas['p1'] + capacitancias_pernas['p2']
        elif rack == 'R2':
            return capacitancias_pernas['p3'] + capacitancias_pernas['p4']

    def get_capacitancia_fase(self, capacitancias_racks):
        C_R1 = capacitancias_racks['R1']
        C_R2 = capacitancias_racks['R2']
        return (C_R1 * C_R2) / (C_R1 + C_R2)

    # Método para calcular o estado do sistema
    def calcular_estados_sistema(self, df: pd.DataFrame = None) -> dict:
        if df is None:
            df = self.df

        # 1. Calcular capacitâncias dos ramos
        ramos = ['A1', 'A2', 'A3', 'A4', 'B1', 'B2', 'B3', 'B4']
        capacitancias_ramos = {ramo: self.get_capacitancia_ramo(df, ramo) for ramo in ramos}

        # 2. Calcular capacitâncias das pernas
        pernas = ['p1', 'p2', 'p3', 'p4']
        capacitancias_pernas = {perna: self.get_capacitancia_perna(capacitancias_ramos, perna) for perna in pernas}

        # 3. Calcular capacitâncias dos racks
        racks = ['R1', 'R2']
        capacitancias_racks = {rack: self.get_capacitancia_rack(capacitancias_pernas, rack) for rack in racks}

        # 4. Calcular capacitância da fase
        capacitancia_fase = self.get_capacitancia_fase(capacitancias_racks)

        # 5. Calcular reatâncias
        X_c_ramos = {ramo: self.get_reatancia_capacitiva_ramo(capacitancias_ramos[ramo]) for ramo in ramos}
        X_c_pernas = {
            'p1': self.get_reatancia_capacitiva_perna(X_c_ramos['A1'], X_c_ramos['A2']),
            'p2': self.get_reatancia_capacitiva_perna(X_c_ramos['B1'], X_c_ramos['B2']),
            'p3': self.get_reatancia_capacitiva_perna(X_c_ramos['A3'], X_c_ramos['A4']),
            'p4': self.get_reatancia_capacitiva_perna(X_c_ramos['B3'], X_c_ramos['B4']),
        }
        X_c_racks = {
            'R1': self.get_reatancia_capacitiva_rack(X_c_pernas['p1'], X_c_pernas['p2']),
            'R2': self.get_reatancia_capacitiva_rack(X_c_pernas['p3'], X_c_pernas['p4']),
        }
        X_c_fase = self.get_reatancia_capacitiva_fase(X_c_racks['R1'], X_c_racks['R2'])

        # 6. Calcular correntes e tensões
        I_fase = self.get_corrente_fase(X_c_fase)
        V_R1 = self.get_tensao_rack(I_fase, X_c_racks['R1'])
        V_R2 = self.get_tensao_rack(I_fase, X_c_racks['R2'])
        I_pernas = {
            'p1': self.get_corrente_perna(V_R1, X_c_pernas['p1']),
            'p2': self.get_corrente_perna(V_R1, X_c_pernas['p2']),
            'p3': self.get_corrente_perna(V_R2, X_c_pernas['p3']),
            'p4': self.get_corrente_perna(V_R2, X_c_pernas['p4']),
        }

        # 7. Calcular corrente de desbalanceamento
        corrente_desbalanceamento = self.get_corrente_desbalanceamento(
            I_pernas['p1'], I_pernas['p2'], I_pernas['p3'], I_pernas['p4']
        )

        return {
            'capacitancias': {
                'ramos': capacitancias_ramos,
                'pernas': capacitancias_pernas,
                'racks': capacitancias_racks,
                'fase': capacitancia_fase
            },
            'reatancias': {
                'ramos': X_c_ramos,
                'pernas': X_c_pernas,
                'racks': X_c_racks,
                'fase': X_c_fase
            },
            'correntes': {
                'fase': I_fase,
                'pernas': I_pernas,
                'desbalanceamento': corrente_desbalanceamento
            },
            'tensoes': {
                'fase': self.V_fase,
                'racks': {
                    'R1': V_R1,
                    'R2': V_R2
                }
            }
        }

    def verificar_restricoes(self, estados: dict) -> bool:
        tolerancia = self.config['constraints']['tolerancia_diferenca']
        corrente_desbalanceamento = estados['correntes']['desbalanceamento']
        return corrente_desbalanceamento <= tolerancia

    def trocar_capacitores(self, idx1: int, idx2: int) -> None:
        temp = self.df.loc[idx1].copy()
        self.df.loc[idx1] = self.df.loc[idx2]
        self.df.loc[idx2] = temp
        self.capacitores_permutados.update([idx1, idx2])

    def otimizar(self) -> dict:
        permutacoes_realizadas = 0

        # Calcular o estado inicial do sistema
        estados_iniciais = self.calcular_estados_sistema(self.df)
        corrente_desbalanceamento_inicial = estados_iniciais['correntes']['desbalanceamento']
        print(f"Corrente de desbalanceamento inicial: {corrente_desbalanceamento_inicial:.6f} A")

        # Obter a tolerância de desbalanceamento entre pernas
        tolerancia_pernas = self.config['constraints']['tolerancia_diferenca_pernas']

        # Iniciar o processo de otimização
        while permutacoes_realizadas < self.max_permutacoes:
            melhoria = False

            # Percorrer os racks
            for rack in self.df['rack'].unique():
                df_rack = self.df[self.df['rack'] == rack]

                # Obter as pernas dentro do rack
                pernas_rack = df_rack['perna'].unique()
                if len(pernas_rack) < 2:
                    continue  # Não há pernas suficientes para balancear

                # Recalcular os estados do sistema atualizados
                estados_atualizados = self.calcular_estados_sistema(self.df)
                capacitancias_pernas = estados_atualizados['capacitancias']['pernas']

                # Identificar as pernas com maior e menor capacitância
                perna_maior = max(pernas_rack, key=lambda p: capacitancias_pernas[p])
                perna_menor = min(pernas_rack, key=lambda p: capacitancias_pernas[p])

                diferenca_pernas = capacitancias_pernas[perna_maior] - capacitancias_pernas[perna_menor]

                if diferenca_pernas > tolerancia_pernas:
                    # Obter capacitores disponíveis para permutação
                    df_perna_maior = df_rack[(df_rack['perna'] == perna_maior) & (~df_rack.index.isin(self.capacitores_permutados))]
                    df_perna_menor = df_rack[(df_rack['perna'] == perna_menor) & (~df_rack.index.isin(self.capacitores_permutados))]

                    if df_perna_maior.empty or df_perna_menor.empty:
                        continue  # Sem capacitores disponíveis

                    # Selecionar capacitores para permutação
                    idx_maior = df_perna_maior['capacitancia_campo_ajustada_uF'].idxmax()
                    idx_menor = df_perna_menor['capacitancia_campo_ajustada_uF'].idxmin()

                    # Realizar a permutação
                    self.trocar_capacitores(idx_maior, idx_menor)
                    permutacao = {
                        'posicao_eln_1': self.df.loc[idx_maior, 'posicao_eln'],
                        'posicao_eln_2': self.df.loc[idx_menor, 'posicao_eln'],
                        'ramo_1': self.df.loc[idx_maior, 'ramo'],
                        'ramo_2': self.df.loc[idx_menor, 'ramo'],
                        'perna_1': self.df.loc[idx_maior, 'perna'],
                        'perna_2': self.df.loc[idx_menor, 'perna'],
                        'rack_1': self.df.loc[idx_maior, 'rack'],
                        'rack_2': self.df.loc[idx_menor, 'rack']
                    }
                    self.permutacoes.append(permutacao)
                    permutacoes_realizadas += 1

                    # Recalcular os estados do sistema após a permutação
                    estados_novos = self.calcular_estados_sistema(self.df)
                    corrente_desbalanceamento_novo = estados_novos['correntes']['desbalanceamento']
                    print(f"Permutação {permutacoes_realizadas}: Corrente de desbalanceamento após permuta: {corrente_desbalanceamento_novo:.6f} A")

                    melhoria = True
                    break  # Sair do loop para reiniciar a verificação após a permutação

            if not melhoria:
                break  # Não há mais permutações possíveis ou necessárias

        # Calcular o estado final do sistema
        estados_finais = self.calcular_estados_sistema(self.df)
        corrente_desbalanceamento_final = estados_finais['correntes']['desbalanceamento']
        print(f"Corrente de desbalanceamento final: {corrente_desbalanceamento_final:.6f} A")
        self.sucesso = self.verificar_restricoes(estados_finais)

        return {
            'sucesso': self.sucesso,
            'corrente_inicial': corrente_desbalanceamento_inicial,
            'corrente_final': corrente_desbalanceamento_final,
            'permutacoes': self.permutacoes,
            'estados_finais': estados_finais,
            'df_otimizado': self.df
        }

In [370]:
# Carregar as configurações
import json

with open('data/config.json', 'r') as file:
    config = json.load(file)

# Definir o número máximo de permutações
max_permutacoes = 5
# Instanciar o otimizador
otimizador = OtimizadorBancoCapacitores(field_measurements_merged, config, max_permutacoes)

# Executar a otimização
resultado = otimizador.otimizar()

# Verificar se a otimização foi bem-sucedida
if resultado['sucesso']:
    print("Otimização realizada com sucesso!")
else:
    print("Otimização não atingiu as restrições com o número máximo de permutações.")

# Exibir as permutações realizadas
print("\nPermutações realizadas:")
for permutacao in resultado['permutacoes']:
    print(f"Troca entre posição {permutacao['posicao_eln_1']} ({permutacao['ramo_1']}, {permutacao['perna_1']}, {permutacao['rack_1']}) "
          f"e posição {permutacao['posicao_eln_2']} ({permutacao['ramo_2']}, {permutacao['perna_2']}, {permutacao['rack_2']})")

# Exibe os parâmetros do sistema após otimização
print("\nParâmetros do sistema após otimização:")
for chave, valor in resultados['parametros'].items():
    if isinstance(valor, dict):
        print(f"{chave}:")
        for subchave, subvalor in valor.items():
            print(f"  {subchave}: {subvalor}")
    else:
        print(f"{chave}: {valor}")

Corrente de desbalanceamento inicial: 0.221969 A
Permutação 1: Corrente de desbalanceamento após permuta: 0.221969 A
Permutação 2: Corrente de desbalanceamento após permuta: 0.221969 A
Permutação 3: Corrente de desbalanceamento após permuta: 0.221969 A
Permutação 4: Corrente de desbalanceamento após permuta: 0.221969 A
Permutação 5: Corrente de desbalanceamento após permuta: 0.221969 A
Corrente de desbalanceamento final: 0.221969 A
Otimização não atingiu as restrições com o número máximo de permutações.

Permutações realizadas:
Troca entre posição 15 (A1, p1, R1) e posição 95 (B2, p2, R1)
Troca entre posição 13 (A2, p1, R1) e posição 90 (B2, p2, R1)
Troca entre posição 5 (A2, p1, R1) e posição 107 (B1, p2, R1)
Troca entre posição 27 (A2, p1, R1) e posição 96 (B2, p2, R1)
Troca entre posição 21 (A2, p1, R1) e posição 117 (B1, p2, R1)

Parâmetros do sistema após otimização:
capacitancias:
  ramos: {'A1': 1.5431268098303932, 'A2': 1.5393547947488146, 'A3': 1.7651594622083038, 'A4': 1.7622

In [86]:
import pandas as pd
import numpy as np

class OtimizadorBancoCapacitores:
    def __init__(self, df: pd.DataFrame, config: dict, rack_to_optimize: str, max_permutacoes: int):
        self.df = df.copy()
        self.config = config
        self.rack_to_optimize = rack_to_optimize
        self.max_permutacoes = max_permutacoes
        self.permutacoes_feitas = []
        self.swapped_capacitors = set()
        self.unbalanced_current = None  # Será calculada
        self.is_balanced = None
        self.result_df = self.df.copy()
        self.constants = config['constants']
        self.constraints = config['constraints']
        
        # Cálculos iniciais
        self.calcular_parametros()
        self.verificar_balanceamento_inicial()
        
        # Se desbalanceado, realizar otimização
        if not self.is_balanced:
            self.otimizar()
    
    def calcular_parametros(self):
        # Calcular capacitâncias
        self.capacitancias_ramos = self.get_capacitancia_ramo()
        self.capacitancias_pernas = self.get_capacitancia_perna()
        self.capacitancias_racks = self.get_capacitancia_racks()
        self.capacitancia_fase = self.get_capacitancia_fase()
        
        # Calcular reatâncias
        self.reatancias_ramos = self.get_reatancia_capacitiva_ramos()
        self.reatancias_pernas = self.get_reatancia_capacitiva_pernas()
        self.reatancias_racks = self.get_reatancia_capacitiva_racks()
        self.reatancia_fase = self.get_reatancia_capacitiva_fase()
        
        # Calcular correntes e tensões
        self.V_fase = self.constants['tensao_nom_kV'] * 1000 / np.sqrt(3)
        self.I_fase = self.get_corrente_fase()
        self.V_racks = self.get_tensao_racks()
        self.I_pernas = self.get_corrente_pernas()
        self.unbalanced_current = self.get_corrente_desbalanceamento()
    
    def verificar_balanceamento_inicial(self):
        tolerancia_diferenca = self.constraints['tolerancia_diferenca']
        corrente_desbalanco_alarme = self.constraints['corrente_desbalanco_alarme_A']
        
        if self.unbalanced_current <= tolerancia_diferenca:
            self.is_balanced = True
            print("O sistema já está balanceado.")
        elif self.unbalanced_current >= corrente_desbalanco_alarme:
            self.is_balanced = False
            print("O sistema não está balanceado e requer otimização.")
        else:
            self.is_balanced = False
            print("O sistema não está balanceado e requer otimização.")
    
    def get_capacitancia_ramo(self):
        branches = self.df['ramo'].unique()
        branches = np.sort(branches)
        
        capacitancias_ramos = {}
        
        for ramo in branches:
            ramo_data = self.result_df[self.result_df['ramo'] == ramo]
            soma_inversos = np.sum(1 / ramo_data['capacitancia_campo_ajustada_uF'].values)
            capacitancias_ramos[ramo] = 1 / soma_inversos
        
        return capacitancias_ramos
    
    def get_capacitancia_perna(self):
        capacitancias_pernas = {
            'P1': self.capacitancias_ramos['A1'] + self.capacitancias_ramos['A2'],
            'P2': self.capacitancias_ramos['B1'] + self.capacitancias_ramos['B2'],
            'P3': self.capacitancias_ramos['A3'] + self.capacitancias_ramos['B4'],
            'P4': self.capacitancias_ramos['B3'] + self.capacitancias_ramos['B4']
        }
        return capacitancias_pernas
    
    def get_capacitancia_racks(self):
        capacitancias_racks = {
            'R1': self.capacitancias_pernas['P1'] + self.capacitancias_pernas['P2'],
            'R2': self.capacitancias_pernas['P3'] + self.capacitancias_pernas['P4']
        }
        return capacitancias_racks
    
    def get_capacitancia_fase(self):
        C_R1 = self.capacitancias_racks['R1']
        C_R2 = self.capacitancias_racks['R2']
        capacitancia_fase = (C_R1 * C_R2) / (C_R1 + C_R2)
        return capacitancia_fase
    
    def get_reatancia_capacitiva_ramos(self):
        frequencia = self.constants['frequencia_Hz']
        phi = 1e-6
        reatancias_ramos = {}
        for ramo, capacitancia in self.capacitancias_ramos.items():
            capacitancia_farad = capacitancia * phi
            reatancia = 1 / (2 * np.pi * frequencia * capacitancia_farad)
            reatancias_ramos[ramo] = reatancia
        return reatancias_ramos
    
    def get_reatancia_capacitiva_pernas(self):
        reatancias_pernas = {}
        for perna in self.capacitancias_pernas:
            if perna == 'P1':
                ramo1, ramo2 = 'A1', 'A2'
            elif perna == 'P2':
                ramo1, ramo2 = 'B1', 'B2'
            elif perna == 'P3':
                ramo1, ramo2 = 'A3', 'A4'
            elif perna == 'P4':
                ramo1, ramo2 = 'B3', 'B4'
            else:
                continue
            Xc_ramo1 = self.reatancias_ramos[ramo1]
            Xc_ramo2 = self.reatancias_ramos[ramo2]
            Xc_perna = (Xc_ramo1 * Xc_ramo2) / (Xc_ramo1 + Xc_ramo2)
            reatancias_pernas[perna] = Xc_perna
        return reatancias_pernas
    
    def get_reatancia_capacitiva_racks(self):
        reatancias_racks = {}
        Xc_P1 = self.reatancias_pernas['P1']
        Xc_P2 = self.reatancias_pernas['P2']
        Xc_R1 = (Xc_P1 * Xc_P2) / (Xc_P1 + Xc_P2)
        reatancias_racks['R1'] = Xc_R1
        Xc_P3 = self.reatancias_pernas['P3']
        Xc_P4 = self.reatancias_pernas['P4']
        Xc_R2 = (Xc_P3 * Xc_P4) / (Xc_P3 + Xc_P4)
        reatancias_racks['R2'] = Xc_R2
        return reatancias_racks
    
    def get_reatancia_capacitiva_fase(self):
        Xc_R1 = self.reatancias_racks['R1']
        Xc_R2 = self.reatancias_racks['R2']
        Xc_fase = Xc_R1 + Xc_R2
        return Xc_fase
    
    def get_corrente_fase(self):
        I_fase = self.V_fase / self.reatancia_fase
        return I_fase
    
    def get_tensao_racks(self):
        V_racks = {}
        V_racks['R1'] = self.I_fase * self.reatancias_racks['R1']
        V_racks['R2'] = self.I_fase * self.reatancias_racks['R2']
        return V_racks
    
    def get_corrente_pernas(self):
        I_pernas = {}
        V_R1 = self.V_racks['R1']
        Xc_P1 = self.reatancias_pernas['P1']
        Xc_P2 = self.reatancias_pernas['P2']
        I_P1 = V_R1 / Xc_P1
        I_P2 = V_R1 / Xc_P2
        V_R2 = self.V_racks['R2']
        Xc_P3 = self.reatancias_pernas['P3']
        Xc_P4 = self.reatancias_pernas['P4']
        I_P3 = V_R2 / Xc_P3
        I_P4 = V_R2 / Xc_P4
        I_pernas['P1'] = I_P1
        I_pernas['P2'] = I_P2
        I_pernas['P3'] = I_P3
        I_pernas['P4'] = I_P4
        return I_pernas
    
    def get_corrente_desbalanceamento(self):
        I_P1 = self.I_pernas['P1']
        I_P2 = self.I_pernas['P2']
        I_P3 = self.I_pernas['P3']
        I_P4 = self.I_pernas['P4']
        I_desb_R1 = np.abs(I_P4 - I_P2)
        I_desb_R2 = np.abs(I_P3 - I_P1)
        I_desbalanceamento = max(I_desb_R1, I_desb_R2)
        return I_desbalanceamento
    
    def otimizar(self):
        num_permutations = 0
        max_permutacoes = self.max_permutacoes
        tolerancia_diferenca = self.constraints['tolerancia_diferenca']
        corrente_desbalanco_alarme = self.constraints['corrente_desbalanco_alarme_A']
        rack = self.rack_to_optimize
        if rack == 'R1':
            legs = ['P1', 'P2']
        elif rack == 'R2':
            legs = ['P3', 'P4']
        else:
            print("Rack inválido.")
            return
        
        leg_branches = {
            'P1': ['A1', 'A2'],
            'P2': ['B1', 'B2'],
            'P3': ['A3', 'A4'],
            'P4': ['B3', 'B4']
        }
        
        leg1_branches = leg_branches[legs[0]]
        leg2_branches = leg_branches[legs[1]]
        
        while self.unbalanced_current > tolerancia_diferenca and num_permutations < max_permutacoes:
            swap_made = False
            for branch1 in leg1_branches:
                capacitors_branch1 = self.result_df[
                    (self.result_df['ramo'] == branch1) &
                    (~self.result_df['posicao_eln'].isin(self.swapped_capacitors))
                ]
                if capacitors_branch1.empty:
                    continue
                cap1 = capacitors_branch1.loc[capacitors_branch1['capacitancia_campo_ajustada_uF'].idxmin()]
                for branch2 in leg2_branches:
                    capacitors_branch2 = self.result_df[
                        (self.result_df['ramo'] == branch2) &
                        (~self.result_df['posicao_eln'].isin(self.swapped_capacitors))
                    ]
                    if capacitors_branch2.empty:
                        continue
                    cap2 = capacitors_branch2.loc[capacitors_branch2['capacitancia_campo_ajustada_uF'].idxmax()]
                    idx1 = cap1.name
                    idx2 = cap2.name
                    pos1 = cap1['posicao_eln']
                    pos2 = cap2['posicao_eln']
                    unbalanced_current_before = self.unbalanced_current
                    temp = self.result_df.loc[idx1, ['perna', 'ramo', 'posicao_eln']]
                    self.result_df.loc[idx1, ['perna', 'ramo', 'posicao_eln']] = self.result_df.loc[idx2, ['perna', 'ramo', 'posicao_eln']]
                    self.result_df.loc[idx2, ['perna', 'ramo', 'posicao_eln']] = temp
                    self.calcular_parametros()
                    if self.unbalanced_current < unbalanced_current_before:
                        self.swapped_capacitors.update([pos1, pos2])
                        self.permutacoes_feitas.append({'from': pos1, 'to': pos2, 'iteration': num_permutations+1})
                        num_permutations +=1
                        print(f"Permutação {num_permutations}: trocou capacitor {pos1} (ramo {branch1}) com {pos2} (ramo {branch2}). Corrente de desbalanceamento reduziu para {self.unbalanced_current:.6f} A")
                        swap_made = True
                        break
                    else:
                        temp = self.result_df.loc[idx1, ['perna', 'ramo', 'posicao_eln']]
                        self.result_df.loc[idx1, ['perna', 'ramo', 'posicao_eln']] = self.result_df.loc[idx2, ['perna', 'ramo', 'posicao_eln']]
                        self.result_df.loc[idx2, ['perna', 'ramo', 'posicao_eln']] = temp
                        self.calcular_parametros()
                if swap_made:
                    break
            if not swap_made:
                print("Nenhuma permutação adicional reduz a corrente de desbalanceamento.")
                break
        print(f"Otimização concluída com {num_permutations} permutações.")
        if self.unbalanced_current <= tolerancia_diferenca:
            print("O sistema está balanceado após otimização.")
        else:
            print("Não foi possível balancear o sistema dentro das tolerâncias com o número máximo de permutações.")
    
    def get_results(self):
        results = {}
        results['is_balanced'] = self.unbalanced_current <= self.constraints['tolerancia_diferenca']
        results['unbalanced_current'] = self.unbalanced_current
        results['permutations'] = self.permutacoes_feitas
        results['capacitancias_ramos'] = self.capacitancias_ramos
        results['capacitancias_pernas'] = self.capacitancias_pernas
        results['capacitancias_racks'] = self.capacitancias_racks
        results['capacitancia_fase'] = self.capacitancia_fase
        results['reatancias_ramos'] = self.reatancias_ramos
        results['reatancias_pernas'] = self.reatancias_pernas
        results['reatancias_racks'] = self.reatancias_racks
        results['reatancia_fase'] = self.reatancia_fase
        results['V_fase'] = self.V_fase
        results['I_fase'] = self.I_fase
        results['V_racks'] = self.V_racks
        results['I_pernas'] = self.I_pernas
        results['updated_dataframe'] = self.result_df
        return results


In [92]:
# Exemplo de uso
# df é o dataframe fornecido
# config é o dicionário de configuração carregado do config.json
# rack_to_optimize é 'R1' ou 'R2'
# max_permutacoes é o número máximo de permutações permitidas

otimizador = OtimizadorBancoCapacitores(field_measurements_merged, config, rack_to_optimize='R1', max_permutacoes=10)
resultados = otimizador.get_results()

# Acessando os resultados
print("Sistema balanceado:", resultados['is_balanced'])
print("Corrente de desbalanceamento após otimização:", resultados['unbalanced_current'])
print("Permutações realizadas:", resultados['permutations'])

# Print other system parameters
print("\nCapacitâncias dos ramos (μF):")
for ramo, cap in resultados['capacitancias_ramos'].items():
    print(f"Ramo {ramo}: {cap}")

print("\nCapacitâncias das pernas (μF):")
for perna, cap in resultados['capacitancias_pernas'].items():
    print(f"Perna {perna}: {cap}")

print("\nCapacitâncias dos racks (μF):")
for rack, cap in resultados['capacitancias_racks'].items():
    print(f"Rack {rack}: {cap}")

print("\nCapacitância da fase (μF):", resultados['capacitancia_fase'])

print("\nReatâncias dos ramos (Ohms):")
for ramo, reatancia in resultados['reatancias_ramos'].items():
    print(f"Ramo {ramo}: {reatancia}")

print("\nReatâncias das pernas (Ohms):")
for perna, reatancia in resultados['reatancias_pernas'].items():
    print(f"Perna {perna}: {reatancia}")

print("\nReatâncias dos racks (Ohms):")
for rack, reatancia in resultados['reatancias_racks'].items():
    print(f"Rack {rack}: {reatancia}")

print("\nReatância da fase (Ohms):", resultados['reatancia_fase'])

print("\nTensão de fase (V):", resultados['V_fase'])
print("Corrente de fase (A):", resultados['I_fase'])

print("\nTensões dos racks (V):")
for rack, tensao in resultados['V_racks'].items():
    print(f"Rack {rack}: {tensao}")

print("\nCorrentes das pernas (A):")
for perna, corrente in resultados['I_pernas'].items():
    print(f"Perna {perna}: {corrente}")


O sistema não está balanceado e requer otimização.
Permutação 1: trocou capacitor 15 (ramo A1) com 107 (ramo B1). Corrente de desbalanceamento reduziu para 0.135464 A
Permutação 2: trocou capacitor 8 (ramo A1) com 117 (ramo B1). Corrente de desbalanceamento reduziu para 0.062181 A
Permutação 3: trocou capacitor 16 (ramo A1) com 93 (ramo B1). Corrente de desbalanceamento reduziu para 0.006670 A
Nenhuma permutação adicional reduz a corrente de desbalanceamento.
Otimização concluída com 3 permutações.
Não foi possível balancear o sistema dentro das tolerâncias com o número máximo de permutações.
Sistema balanceado: False
Corrente de desbalanceamento após otimização: 0.0066700175322580435
Permutações realizadas: [{'from': 15, 'to': 107, 'iteration': 1}, {'from': 8, 'to': 117, 'iteration': 2}, {'from': 16, 'to': 93, 'iteration': 3}]

Capacitâncias dos ramos (μF):
Ramo A1: 1.5466303692086558
Ramo A2: 1.5393547947488146
Ramo A3: 1.7651594622083038
Ramo A4: 1.7622254945638736
Ramo B1: 1.541223

In [90]:
print("DataFrame atualizado:")
print(resultados['updated_dataframe'])

DataFrame atualizado:
               date_time rack ramo perna  posicao_eln  num_serie  \
0    22/12/2020 08:09:18   R1   A1    p1            1        923   
1    22/12/2020 08:10:12   R1   A1    p1            2        925   
2    22/12/2020 08:10:48   R1   A2    p1            3        924   
3    22/12/2020 08:11:24   R1   A2    p1            4        919   
4    22/12/2020 08:12:20   R1   A2    p1            5        557   
..                   ...  ...  ...   ...          ...        ...   
115  22/12/2020 09:43:53   R1   B1    p2          116       1100   
116  22/12/2020 09:44:39   R1   A1    p1            8       1089   
117  22/12/2020 09:45:10   R1   B1    p2          118       1070   
118  22/12/2020 09:45:52   R1   B2    p2          119       1073   
119  22/12/2020 09:46:28   R1   B2    p2          120       1076   

     capacitancia_fabrica_uF  delta_capacitancia_fabrica_%  \
0                      24.87                      2.345679   
1                      24.88         

In [ ]:
v